In [1]:
import random
import pandas as pd
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\ramir\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [2]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,8):
    url = "https://simple.ripley.cl/tecno/computacion/notebooks?page="+str(i)
    url_base.append(url)

In [3]:
##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="catalog-product-item catalog-product-item__container col-xs-6 col-sm-6 col-md-4 col-lg-4"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)

In [4]:
##Obtención de datos dentro de cada una de las páginas del sitio web, almacenados en listas auxiliares


nombre_equipo = []
precios = []
almacenamiento = []
procesador = []
ram = []
codigos = []

In [5]:
for i in url_notebooks:
    nombres = []
    valores = []        
    driver.get(i)
    sleep(5)
    boton = driver.find_element_by_xpath('//div[@aria-controls="panel-Especificaciones"]').click()
    sleep(2)
    try:
        ver = driver.find_element_by_xpath('//button[@class="btn-loading  rpl-show-more btn-outline-grey "]').click()
    except:
        x=2
    bloque = driver.find_elements_by_xpath('//tr')
    bloque_nombre = driver.find_elements_by_xpath('//section[@class="product-header hidden-xs"]')
    try:
        precio = driver.find_element_by_xpath('//dt[@class="product-price"]')
    except:
        precio = driver.find_element_by_xpath('//dt[@class="product-price product-price__line-thru"]')
    for i in bloque_nombre:
        nombre = i.find_element_by_xpath('.//h1')
        codigo = i.find_element_by_xpath('.//span[@class="sku sku-value"]')
    for i in bloque:
        elementos = i.find_elements_by_xpath('.//td')
        for j in range(len(elementos)):
            if (j==0):
                nombres.append(elementos[j].text)
            if (j==1):
                valores.append(elementos[j].text)
                
    if ("Tipo Procesador" in nombres)==True:
        index_procesador = nombres.index("Tipo Procesador")
        procesador.append(valores[index_procesador])

    if ("Tipo Procesador" in nombres)==False:
        procesador.append("Procesador no especificado")

    if ("Memoria Ram" in nombres)==True:
        index_ram = nombres.index("Memoria Ram")
        
        ram.append(valores[index_ram])
        
    if ("Memoria Ram" in nombres)==False:
        ram.append("Memoria Ram no especificada")
        
    if ("Capacidad Disco Duro" in nombres)==True:
        idex_almacenamiento = nombres.index("Capacidad Disco Duro")
        almacenamiento.append(valores[idex_almacenamiento])
        
    if ("Capacidad Disco Duro" in nombres)==False:
        almacenamiento.append("Almacenamiento no especificado")
        
    codigos.append(codigo.text)
    precios.append(precio.text)
    nombre_equipo.append(nombre.text)

In [6]:
for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")

In [7]:
df = pd.DataFrame(columns=['Nombre','Código', 'Almacenamiento', 'Procesador', 'Memoria Ram', 'Precio'])

In [8]:
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i], 'Código':codigos[i],'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i]},ignore_index=True)

In [9]:
df['Nombre'] = df['Nombre'].astype(str)
df['Almacenamiento'] = df['Almacenamiento'].astype(str)
df['Código'] = df['Código'].astype(str)
df['Procesador'] = df['Procesador'].astype(str)
df['Memoria Ram'] = df['Memoria Ram'].astype(str)
df['Precio'] = df['Precio'].astype(int)

In [10]:
df

,Nombre,Código,Almacenamiento,Procesador,Memoria Ram,Precio
0,HP LAPTOP 15-EF1018LA / AMD RYZEN™ 5 / 8GB RAM...,2000383107424,256 GB SSD,AMD Ryzen™ 5,8 GB,499990
1,HP LAPTOP 14-DQ2026LA / INTEL® CORE™ I3 / 20GB...,2000385847632,256 GB SSD,Intel Core i3,4 GB,429990
2,HP PAVILION GAMING LAPTOP 15-EC1024LA / AMD RY...,2000381137980,512 GB SSD,AMD Ryzen™ 7,8 GB,799990
3,HP PAVILION GAMING LAPTOP 15-EC1026LA / AMD RY...,2000379735440,256 GB SSD,AMD Ryzen™ 5,8 GB,699990
4,ACER NITRO 5 / INTEL® CORE™ I5 / 16GB RAM / 51...,2000383871721,512 GB SSD,Intel Core i5,16 GB,849990
...,...,...,...,...,...,...
219,HP PROBOOK 450 440 445 455 470 G0 G1 NOTEBOOK ...,MPM00012599680,Almacenamiento no especificado,Procesador no especificado,Memoria Ram no especificada,56900
220,MSI GF65 THIN GAMING CORE™ I7 512GB SSD 32GB R...,MPM00015353173,512 GB SSD,Intel Core i7,32 GB,1339000
221,NOTEBOK GAMER ASUS ROG STRIX I7-10750 16GB 1TB...,MPM00016416098,1 TB SSD,Intel Core i7,16 GB,1799990
222,NOTEBOOK GAMER ASUS ROG ZEPHYRUS G15 AMD RYZEN...,MPM00016416390,1 TB SSD,AMD Ryzen,16 GB,1849990
